### Perspective API Exploration

First, we have a dataset of Wikipedia comments made available by Jigsaw, a subsidiary of Google that created the Perspective tool. The dataset includes a unique comment id, the text of the comment, and a series of binary labels applied by human raters: "toxic," "severe_toxic," "obscene," "threat," "insult," and "identity_hate." I have appended the "score" column, which represents the toxicity score assigned to the comment text by the live version of the Perspective API. The data is available under a CC0 license.

In [1]:
import pandas as pd
import time

df = pd.read_csv('labeled_and_scored_comments.csv')

In [2]:
df.sort_values(['score'])

,id,comment_text,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
1158,61a85266de70cca0,Можешь говорить по русски.,0.000004,0,0,0,0,0,0
5246,030a9ba0b32b4456,Rodzina Krusenstern Potomkowie Admirała Adama ...,0.000009,0,0,0,0,0,0
11040,a6814193ac6b4705,The opening paragraph states that the game is ...,0.000091,0,0,0,0,0,0
24192,f0d6dee96e057a5b,The book is used as a source in a number of ar...,0.000116,0,0,0,0,0,0
2332,cbb9268d3c7429c1,Vals Vienés \n |Rumba \n |Cha Cha Chá(ciclo 7),0.000131,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
19994,1429a2c930688317,you fucking piece of shit,0.997278,1,1,1,0,1,0
28728,d82e45eb103ad07b,fuck off you stupid aspy asshole,0.997736,1,1,1,0,1,0
25535,e9b92c6c6b01aeef,FUCK YOU YOU PIECE OF SHIT \n FUCK YOU YOU PIE...,0.997982,1,1,1,0,1,0
25945,e783fd267f3a9d3b,FUCK WIKIPEDIA ON WHEELS! \n\nFuck off wikiped...,0.998136,1,1,1,0,1,0


Data Exploration:
The threshold that I am using to determine what comments are toxic is .45 or 45%. This is a little under the half way mark which indicates the comment is pretty much neutral. Anything greater than the neutral half way threshold should be considered "toxic". The labels I will be looking at to determine if comments are toxic are the "toxic" and "severe toxic" since that is what I will be testing the score for. 

In [7]:
from googleapiclient.discovery import build
import json

def get_toxicity_score(comment):
    
  API_KEY = 'xxxxxxxxxxxxxxx' # Put your API key here
    
  client = build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1"
  )

  analyze_request = {
  'comment': { 'text': comment },
  'requestedAttributes': {'TOXICITY': {}}
  }
    
  response = client.comments().analyze(body=analyze_request).execute()
  toxicity_score = response["attributeScores"]["TOXICITY"]["summaryScore"]["value"]
    
  return toxicity_score

Hypothesis:
The API tends to mark homosexual terms as more toxic than they actually are intended to be. 

In [21]:
get_toxicity_score("gay")

0.61826205

In [22]:
get_toxicity_score("lesbian")

0.502098

In [23]:
get_toxicity_score("queer")

0.26817688

In [24]:
get_toxicity_score("homo")

0.53430706

In [25]:
get_toxicity_score("homosexual")

0.49981686

In [27]:
get_toxicity_score("bisexual")

0.3282253

In [28]:
comment_list = ['gay', 'queer', 'homo']
score_list = []

for comment in comment_list:
    score = get_toxicity_score(comment)
    score_list.append(score)
    time.sleep(1)

In [29]:
sum(score_list)

1.42074599

Analyze Data & Results:

I predicted that there would be biases in the system of how homosexual terms would be percieved by the API. This is due to the inherent stigma around homosexuality in society and how it is still not considered to be fully "normal". Though homosexuality is being more accepted amongst a lot of people, the API does not seem to follow the trends of society. Neutral terms such as gay, homo, and lesbian could be considered toxic by the API when used in normal ways such as queer awareness and education. These terms could also be used in a negative way but overall homosexual terms should not be considered toxic. 

The toxicity scores of homosexual comments tend to be more false positive. There are many cases where the words gay, homo, lesbian etc., are used in a non-toxic way but they all still tend to get toxicity scores of around .5 or above .5 which is in my opinion highly skewed. Most of these scores are above .45 which is considered as toxic. Overall I disagree with how the API perceives how toxic homosexual terms are. 

My theory about these results are the API is not up to date with how society views homosexuality today. There is still a lot of work done to fully accept homosexuality, but in general it is not something that should be deemed as toxic as the API regeisters these terms to be. The API is definitely biased against homosexual terms and marks these normal terms to be more toxic than they actually are. My hypothesis is proven true through this data analysis. 
 